### Import

In [209]:
# 데이터 처리 
import pandas as pd
import numpy as np
# 시각화
import matplotlib.pyplot as plt
import seaborn as sns
import folium
# 파일 관리 
import os
# 경고 메세지 제거
import warnings
warnings.filterwarnings("ignore")

## EDA

### Read Data

`-` 데이터 확인하기.

In [210]:
os.getcwd()

'C:\\Users\\oldcar\\Documents\\GitHub\\2022-AnalysisCompetition-Subway'

In [211]:
for path, _, files in os.walk(os.getcwd() + '/Data'): # mac에선 /
    for filename in files:
        print(os.path.join(path, filename))

C:\Users\oldcar\Documents\GitHub\2022-AnalysisCompetition-Subway/Data\Found Data\공기질_승객수_통합.csv
C:\Users\oldcar\Documents\GitHub\2022-AnalysisCompetition-Subway/Data\Found Data\서울교통공사_역사건축정보_20200331.csv
C:\Users\oldcar\Documents\GitHub\2022-AnalysisCompetition-Subway/Data\Found Data\서울교통공사_역사심도정보_20200929.csv
C:\Users\oldcar\Documents\GitHub\2022-AnalysisCompetition-Subway/Data\Found Data\서울교통공사_월별승하차인원_20201231.csv
C:\Users\oldcar\Documents\GitHub\2022-AnalysisCompetition-Subway/Data\Found Data\서울교통공사_환기구정보_202006.csv
C:\Users\oldcar\Documents\GitHub\2022-AnalysisCompetition-Subway/Data\Found Data\좌표.csv
C:\Users\oldcar\Documents\GitHub\2022-AnalysisCompetition-Subway/Data\Given Data\aircondition_2020.xlsx
C:\Users\oldcar\Documents\GitHub\2022-AnalysisCompetition-Subway/Data\Given Data\data1.csv
C:\Users\oldcar\Documents\GitHub\2022-AnalysisCompetition-Subway/Data\Given Data\data2.csv
C:\Users\oldcar\Documents\GitHub\2022-AnalysisCompetition-Subway/Data\Given Data\data3.csv
C:\Users\

`-` 데이터 가져오기. 

In [212]:
congestion=pd.read_csv('Data/Given Data/data1.csv') # 지하철 혼잡도 데이터
aircondition = pd.read_csv('Data/Given Data/data2.csv') # 지하철 공기질 데이터 
station = pd.read_csv('Data/Given Data/data3.csv') # 지하철 정보 데이터
passengers = pd.read_csv('Data/Given Data/passengers2020.csv') # 지하철 별 승객 수 데이터

### Preprocessing

#### congestion

In [213]:
congestion=pd.read_csv('Data/Given Data/data1.csv')
congestion.head() #전처리 필요없음. 

,조사일자,호선,역번호,역명,구분,5:30,6:00,6:30,7:00,7:30,...,20:00,20:30,21:00,21:30,22:00,22:30,23:00,23:30,24:00:00,24:30:00
0,평일,1호선,150,서울역,상선,8.2,12.9,21.5,28.4,46.7,...,15.1,15.9,17.1,16.0,19.0,15.2,10.5,7.8,6.0,0.9
1,평일,1호선,150,서울역,하선,11.8,13.5,12.2,17.8,26.4,...,36.5,31.2,37.6,44.6,45.6,37.8,28.8,19.3,7.1,0.4
2,평일,1호선,151,시청,상선,7.7,11.2,19.6,24.4,36.2,...,18.7,18.3,20.7,22.0,24.0,18.7,13.7,10.0,6.9,1.0
3,평일,1호선,151,시청,하선,9.5,11.3,11.6,16.7,23.2,...,32.0,27.7,34.8,39.3,37.4,36.3,23.0,18.1,4.0,1.6
4,평일,1호선,152,종각,상선,6.3,8.1,15.5,16.6,22.7,...,25.8,24.2,30.0,31.9,34.0,26.8,18.8,12.5,9.1,1.2


In [214]:
print(congestion.isnull().sum(), congestion.shape) #널값이 있지만 해당 시각에 운행하지 않은 정보도 필요

조사일자           0
호선             0
역번호            0
역명             0
구분             0
5:30          32
6:00          10
6:30          10
7:00          10
7:30          10
8:00          10
8:30          10
9:00          10
9:30          10
10:00         10
10:30         10
11:00         10
11:30         10
12:00         10
12:30         10
13:00         10
13:30         10
14:00         10
14:30         10
15:00         10
15:30         10
16:00         10
16:30         10
17:00         10
17:30         10
18:00         10
18:30         10
19:00         10
19:30         10
20:00         10
20:30         10
21:00         10
21:30         10
22:00         10
22:30         10
23:00         10
23:30         22
24:00:00    1112
24:30:00    1112
dtype: int64 (1668, 44)


#### aircondition

In [215]:
aircondition.head()

,호선,역사명,미세먼지(PM10),이산화탄소(CO2),포름알데히드(HCHO),일산화탄소(CO)
0,1,서울역1,76.4,627,16.4,0.6
1,1,시청1,100.7,610,17.8,1.0
2,1,종각,99.8,687,20.5,0.9
3,1,종로3가1,105.3,646,20.6,1.3
4,1,종로5가,99.4,532,16.4,1.1


공기질 점수를 만들어야 하니 가중합을 사용해야한다. 

In [216]:
aircondition.drop_duplicates(['호선'])['호선'] #중복 제거 후 값 확인

0      1
10     2
47     3
79     4
100    5
151    6
189    7
237    8
Name: 호선, dtype: int64

In [217]:
line_num = [f'{i}' for i in range(1,9)]

# 역사명 뒤에 붙은 호선 제거

for i in range(len(aircondition)):
    if aircondition.loc[i]['역사명'][-1] in line_num:
        aircondition.loc[i,'역사명'] = aircondition.loc[i,'역사명'][:-1]
        
# 호선 값 뒤에 '호선' 붙이기

aircondition['호선'] = aircondition['호선'].apply(lambda x : str(x) + '호선')

# 열 이름 변경 

aircondition.columns = ['호선', '역명', '미세먼지', '이산화탄소', '포름알데히드', '일산화탄소']

# 공기질 점수 0~1 사이로 표준화 (minmax scaler)

score_df = aircondition.loc[:,'미세먼지':'일산화탄소']

aircondition.loc[:,'미세먼지':'일산화탄소'] = score_df.apply(lambda x : (x - x.min()) / (x.max() - x.min()), axis = 0)
aircondition['공기질점수'] = aircondition.loc[:,'미세먼지':'일산화탄소'].mean(axis = 1)
aircondition['공기질점수'] = (aircondition['공기질점수'] - aircondition['공기질점수'].min()) / (aircondition['공기질점수'].max() - aircondition['공기질점수'].min())

aircondition.head()


,호선,역명,미세먼지,이산화탄소,포름알데히드,일산화탄소,공기질점수
0,1호선,서울역,0.335559,0.623145,0.371166,0.166667,0.408431
1,1호선,시청,0.741235,0.572700,0.414110,0.500000,0.700221
2,1호선,종각,0.726210,0.801187,0.496933,0.416667,0.785164
3,1호선,종로3가,0.818030,0.679525,0.500000,0.750000,0.907447
4,1호선,종로5가,0.719533,0.341246,0.371166,0.583333,0.615350


#### Station

In [218]:
station = pd.read_csv('Data/Given Data/data3.csv') # 지하철 정보 데이터
station.head()

,연번,자치구,해당역(호선),역개수
0,1,강남구,삼성(2) 선릉(2) 역삼(2) 강남(2) 압구정(3) 신사(3) 매봉(3) 도곡(...,21
1,2,강동구,천호(5) 강동(5) 길동(5) 굽은다리(5) 명일(5) 고덕(5) 상일동(5) 둔...,14
2,3,강북구,수유(4) 미아(4) 미아사거리(4),3
3,4,강서구,방화(5) 개화산(5) 김포공항(5) 송정(5) 마곡(5) 발산(5) 우장산(5) ...,9
4,5,관악구,낙성대(2) 서울대입구(2) 봉천(2) 신림(2),4


In [219]:
# 연번 제거 

station.drop(['연번'], axis = 1, inplace = True) # inplace 원본변화

# 각 구에 속하는 역 쪼개기

station_ls = []
gu_ls= []

for i in range(len(station)):
    station_split = [x for x in station['해당역(호선)'].str.split()[i]]
    station_ls += station_split
    gu_ls += [f'{station["자치구"][i]}']* len(station_split)

line = [f'{i[-2]}호선' for i in station_ls]

# 데이터프레임으로 다시 만들기. 

station_ls = list(map(lambda x : x[:-3],station_ls))
station = pd.DataFrame({'호선' : line, '역명' : station_ls, '자치구' : gu_ls})

# 9호선 제거 (8호선까지 데이터가 있음)

line9_mask = station['호선'] != '9호선'
station = station[line9_mask]

station.head()

,호선,역명,자치구
0,2호선,삼성,강남구
1,2호선,선릉,강남구
2,2호선,역삼,강남구
3,2호선,강남,강남구
4,3호선,압구정,강남구


#### Passengers

In [220]:
passengers = pd.read_csv('Data/Given Data/passengers2020.csv')
passengers.head()

,연번,호선,역번호,역명,2020년1월,2020년2월,2020년3월,2020년4월,2020년5월,2020년6월,2020년7월,2020년8월,2020년9월,2020년10월,2020년11월,2020년12월
0,1,1,150,서울역(1),3198290.0,2271168.0,1601396.0,1803550.0,2112629.0,2201148.0,2391615.0,1986290.0,1705059.0,2121997.0,2218093.0,1648262.0
1,2,1,151,시청(1),1477362.0,1149641.0,888317.0,874072.0,962697.0,1042088.0,1134098.0,922958.0,853911.0,994423.0,1072922.0,857434.0
2,3,1,152,종각,2462560.0,1945843.0,1475933.0,1586710.0,1745454.0,1892509.0,2069902.0,1626359.0,1378130.0,1702997.0,1758055.0,1314449.0
3,4,1,153,종로3가(1),1935106.0,1421415.0,1095035.0,1224282.0,1374997.0,1360814.0,1523797.0,1212158.0,1051916.0,1298918.0,1297683.0,953894.0
4,5,1,154,종로5가,1527217.0,1149021.0,947157.0,1040487.0,1182917.0,1182427.0,1263489.0,1017196.0,988661.0,1124400.0,1100466.0,930913.0


In [221]:
# 분석의 용이성을 위해 9호선은 제거하였음
line_9_mask = passengers['호선'] != 9
passengers = passengers[line_9_mask]

# 호선을 ?호선으로 변경
passengers['호선'] = passengers['호선'].apply(lambda x : f"{x}호선")

# 역명 뒤에 붙은 (호선)을 제거
passengers['역명'] = passengers['역명'].apply(lambda x : x[:-3] if x[-1] == ')' else x) 

# 연번 제거
passengers.drop(['연번'], axis = 1, inplace = True)

# 열 이름 변경
passengers.columns = ['호선', '역번호', '역명'] + [f"{i}월" for i in range(1, 13)]


# 평균 승객수 
passengers['승객수'] = passengers.loc[:, '1월':'12월'].mean(axis = 1)

# Min Max Scaler
passengers['승객수'] = (passengers['승객수'] - passengers['승객수'].min()) / (passengers['승객수'].max() - passengers['승객수'].min())

passengers.head()

,호선,역번호,역명,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월,승객수
0,1호선,150,서울역,3198290.0,2271168.0,1601396.0,1803550.0,2112629.0,2201148.0,2391615.0,1986290.0,1705059.0,2121997.0,2218093.0,1648262.0,0.478710
1,1호선,151,시청,1477362.0,1149641.0,888317.0,874072.0,962697.0,1042088.0,1134098.0,922958.0,853911.0,994423.0,1072922.0,857434.0,0.225062
2,1호선,152,종각,2462560.0,1945843.0,1475933.0,1586710.0,1745454.0,1892509.0,2069902.0,1626359.0,1378130.0,1702997.0,1758055.0,1314449.0,0.394990
3,1호선,153,종로3가,1935106.0,1421415.0,1095035.0,1224282.0,1374997.0,1360814.0,1523797.0,1212158.0,1051916.0,1298918.0,1297683.0,953894.0,0.293588
4,1호선,154,종로5가,1527217.0,1149021.0,947157.0,1040487.0,1182917.0,1182427.0,1263489.0,1017196.0,988661.0,1124400.0,1100466.0,930913.0,0.248899


#### station 추가작업

In [222]:
# station 테이블에 역번호 추가. 
print(station.head(), '\n', passengers.loc[:,'호선':'역명'].head())
station = pd.merge(station, passengers.loc[:,'호선':'역명'], how = 'right', on = ['호선', '역명'])
station

    호선   역명  자치구
0  2호선   삼성  강남구
1  2호선   선릉  강남구
2  2호선   역삼  강남구
3  2호선   강남  강남구
4  3호선  압구정  강남구 
     호선  역번호    역명
0  1호선  150   서울역
1  1호선  151    시청
2  1호선  152    종각
3  1호선  153  종로3가
4  1호선  154  종로5가


,호선,역명,자치구,역번호
0,1호선,서울역,NaN,150
1,1호선,시청,중구,151
2,1호선,종각,종로구,152
3,1호선,종로3가,종로구,153
4,1호선,종로5가,종로구,154
...,...,...,...,...
275,8호선,남한산성입구,성남시,2823
276,8호선,단대오거리,성남시,2824
277,8호선,신흥,성남시,2825
278,8호선,수진,성남시,2826


In [223]:
# 결측치 처리 
print(station[pd.isnull(station).loc[:,'자치구']])

      호선      역명  자치구   역번호
0    1호선     서울역  NaN   150
110  4호선     서울역  NaN   426
116  4호선  이수(총신대  NaN   432
235  7호선      논현  NaN  2734
263  8호선      암사  NaN  2811


In [224]:
station.loc[station['역명'] == '서울역','자치구'] = "중구"  # loc 함수는 행, 열 인덱싱을 통해 정보를 뽑을 수도 있지만 조건을 넣어 추출할 수도 있다. 
                                                          # station.loc[station['역명'] == '서울역']['자치구'] = '중구' 이건 또 안됐음
station.loc[station['역명'] == "이수(총신대", '역명'] = "이수(총신대입구)"
station.loc[station['역명'] == "이수(총신대입구)",'자치구'] = "동작구"
station.loc[station['역명'] == "암사","자치구"] = "강동구"
station.loc[station['역명'] == "논현","자치구"] = "강남구"

In [225]:
# 자치구 범위 설정

gu_list = ['도봉구', '노원구', '강북구', '은평구', '종로구', '성북구', '중랑구', '서대문구', 
           '동대문구', '강서구', '마포구', '중구', '성동구', '광진구', '강동구', '영등포구',
          '용산구', '양천구',' 구로구', '동작구', '송파구', '강남구', '서초구', '관악구', '금천구']

station = station[station['자치구'].isin(gu_list)]
station.head()

,호선,역명,자치구,역번호
0,1호선,서울역,중구,150
1,1호선,시청,중구,151
2,1호선,종각,종로구,152
3,1호선,종로3가,종로구,153
4,1호선,종로5가,종로구,154


#### congestion 추가작업

In [226]:
# station 정보 없는 conjestion 삭제 
congestion = pd.merge(station, congestion, how = 'inner', on = ['역번호'])
congestion.columns

Index(['호선_x', '역명_x', '자치구', '역번호', '조사일자', '호선_y', '역명_y', '구분', '5:30',
       '6:00', '6:30', '7:00', '7:30', '8:00', '8:30', '9:00', '9:30', '10:00',
       '10:30', '11:00', '11:30', '12:00', '12:30', '13:00', '13:30', '14:00',
       '14:30', '15:00', '15:30', '16:00', '16:30', '17:00', '17:30', '18:00',
       '18:30', '19:00', '19:30', '20:00', '20:30', '21:00', '21:30', '22:00',
       '22:30', '23:00', '23:30', '24:00:00', '24:30:00'],
      dtype='object')

In [227]:
# 결측치 있는 열과 중복되는 열 제거
congestion = pd.concat([congestion.loc[:,['호선_x', '역명_x', '역번호', '자치구', '조사일자', '구분']] , congestion.loc[:, '7:00':'23:00']], axis = 1)
congestion.dropna(axis = 0, inplace=True)

# 열 이름 변경 

congestion.columns = ['호선', '역명'] + list(congestion.columns[2:])

congestion.head()

,호선,역명,역번호,자치구,조사일자,구분,7:00,7:30,8:00,8:30,...,18:30,19:00,19:30,20:00,20:30,21:00,21:30,22:00,22:30,23:00
0,1호선,서울역,150,중구,평일,상선,28.4,46.7,70.0,74.5,...,42.7,29.8,21.7,15.1,15.9,17.1,16.0,19.0,15.2,10.5
1,1호선,서울역,150,중구,평일,하선,17.8,26.4,34.8,25.2,...,76.5,51.0,37.4,36.5,31.2,37.6,44.6,45.6,37.8,28.8
2,1호선,서울역,150,중구,토요일,상선,14.4,15.9,24.6,27.1,...,23.3,18.6,18.3,15.3,16.5,15.9,16.8,18.9,14.4,12.5
3,1호선,서울역,150,중구,토요일,하선,13.8,9.7,16.1,16.5,...,40.2,37.8,33.7,28.0,29.2,34.8,38.0,41.1,29.5,27.4
4,1호선,서울역,150,중구,일요일,상선,7.9,10.3,19.2,16.4,...,14.6,13.2,11.0,14.2,11.8,11.0,16.2,14.7,8.8,7.6


In [228]:
# 조사일자를 평일과 주말로 변경
congestion.loc[congestion['조사일자'] != '평일','조사일자'] = '주말'

# 외선, 내선을 상선과 하선으로 변경(분석의 용이성을 위하여)
congestion.loc[congestion['구분'] == '외선', '구분'] = '상선'
congestion.loc[congestion['구분'] == '내선', '구분'] = '하선'

congestion

,호선,역명,역번호,자치구,조사일자,구분,7:00,7:30,8:00,8:30,...,18:30,19:00,19:30,20:00,20:30,21:00,21:30,22:00,22:30,23:00
0,1호선,서울역,150,중구,평일,상선,28.4,46.7,70.0,74.5,...,42.7,29.8,21.7,15.1,15.9,17.1,16.0,19.0,15.2,10.5
1,1호선,서울역,150,중구,평일,하선,17.8,26.4,34.8,25.2,...,76.5,51.0,37.4,36.5,31.2,37.6,44.6,45.6,37.8,28.8
2,1호선,서울역,150,중구,주말,상선,14.4,15.9,24.6,27.1,...,23.3,18.6,18.3,15.3,16.5,15.9,16.8,18.9,14.4,12.5
3,1호선,서울역,150,중구,주말,하선,13.8,9.7,16.1,16.5,...,40.2,37.8,33.7,28.0,29.2,34.8,38.0,41.1,29.5,27.4
4,1호선,서울역,150,중구,주말,상선,7.9,10.3,19.2,16.4,...,14.6,13.2,11.0,14.2,11.8,11.0,16.2,14.7,8.8,7.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1489,8호선,복정,2821,송파구,평일,하선,12.7,15.5,27.1,18.5,...,49.9,38.8,32.1,26.9,27.0,26.8,28.4,30.1,31.8,24.2
1490,8호선,복정,2821,송파구,주말,상선,17.5,25.4,40.9,29.1,...,34.0,21.4,20.7,15.7,17.5,16.2,13.4,17.0,13.4,15.4
1491,8호선,복정,2821,송파구,주말,하선,6.6,6.7,11.4,9.2,...,22.9,26.4,24.4,19.3,22.0,20.3,22.8,24.5,26.0,24.4
1492,8호선,복정,2821,송파구,주말,상선,9.6,15.0,20.7,22.2,...,18.2,23.1,25.9,12.9,17.6,14.2,12.5,10.2,12.7,8.4


In [229]:
# 토, 일 주말 데이터 묶기 (평균)
congestion_df = congestion.groupby(['역명', '호선', '구분', '조사일자']).mean().reset_index()

#역번호 int형으로 전환
congestion_df['역번호'] = congestion_df['역번호'].astype('int')

congestion_df

,역명,호선,구분,조사일자,역번호,7:00,7:30,8:00,8:30,9:00,...,18:30,19:00,19:30,20:00,20:30,21:00,21:30,22:00,22:30,23:00
0,가락시장,3호선,상선,주말,340,5.25,6.00,9.75,9.85,14.80,...,7.85,6.65,7.15,6.65,5.70,6.05,6.80,12.8,5.70,4.05
1,가락시장,3호선,상선,평일,340,20.50,34.20,39.50,27.50,18.40,...,14.80,9.70,6.90,6.00,6.50,6.00,6.10,6.7,5.50,4.40
2,가락시장,3호선,하선,주말,340,1.80,1.90,3.70,5.15,5.05,...,8.65,6.55,5.90,5.55,7.50,6.55,8.30,6.9,9.50,9.90
3,가락시장,3호선,하선,평일,340,3.60,6.20,11.30,11.60,5.00,...,22.80,16.80,11.70,10.20,10.80,8.60,10.60,9.7,12.50,9.80
4,가락시장,8호선,상선,주말,2818,17.35,24.45,33.85,34.30,44.80,...,43.35,35.30,35.45,31.10,26.05,27.40,25.85,33.5,22.65,20.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,회현,4호선,하선,평일,425,24.90,33.30,45.50,42.00,37.30,...,50.10,37.10,28.30,31.50,35.80,35.00,37.90,48.2,36.50,31.70
988,효창공원앞,6호선,상선,주말,2628,7.20,8.55,12.25,13.30,14.00,...,30.25,26.40,23.00,20.15,20.10,24.10,25.10,28.4,27.85,29.10
989,효창공원앞,6호선,상선,평일,2628,13.70,25.00,41.70,35.80,29.40,...,53.80,46.50,36.30,30.40,29.70,30.10,35.50,36.5,33.10,32.20
990,효창공원앞,6호선,하선,주말,2628,11.75,11.30,17.80,18.25,22.10,...,27.90,21.90,20.65,18.60,18.50,18.70,19.60,19.8,18.35,18.30


In [230]:
# 주말 데이터가 존재하지 않는 역 제외 (주말이 존재하지 않은 열이 있느가?에 대한 정보는 감으로 얻음)
df = congestion_df.groupby(['역번호']).count().reset_index()
df.loc[a['역명'] != 4,'역번호']

166    2612
167    2613
168    2614
169    2616
Name: 역번호, dtype: int64

In [231]:
congestion_df =congestion_df[-congestion_df['역번호'].isin(df.loc[df['역명'] != 4,'역번호'])]

In [232]:
# 자치구 생성

congestion_df = pd.merge(station, congestion_df, on = ['역번호', '역명', '호선'])

In [233]:
congestion_df

,호선,역명,자치구,역번호,구분,조사일자,7:00,7:30,8:00,8:30,...,18:30,19:00,19:30,20:00,20:30,21:00,21:30,22:00,22:30,23:00
0,1호선,서울역,중구,150,상선,주말,11.15,13.10,21.90,21.75,...,18.95,15.90,14.65,14.75,14.15,13.45,16.50,16.80,11.60,10.05
1,1호선,서울역,중구,150,상선,평일,28.40,46.70,70.00,74.50,...,42.70,29.80,21.70,15.10,15.90,17.10,16.00,19.00,15.20,10.50
2,1호선,서울역,중구,150,하선,주말,10.45,7.55,12.60,14.05,...,32.35,31.25,27.60,22.70,24.65,28.05,30.60,32.30,21.95,20.00
3,1호선,서울역,중구,150,하선,평일,17.80,26.40,34.80,25.20,...,76.50,51.00,37.40,36.50,31.20,37.60,44.60,45.60,37.80,28.80
4,1호선,시청,중구,151,상선,주말,9.85,11.40,18.60,18.80,...,20.70,17.20,15.80,17.10,16.65,15.30,18.20,19.30,14.80,11.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975,8호선,장지,송파구,2820,하선,평일,31.50,39.80,55.50,34.70,...,76.80,56.50,44.80,38.90,38.00,41.30,42.70,44.10,44.10,31.90
976,8호선,복정,송파구,2821,상선,주말,13.55,20.20,30.80,25.65,...,26.10,22.25,23.30,14.30,17.55,15.20,12.95,13.60,13.05,11.90
977,8호선,복정,송파구,2821,상선,평일,37.90,56.80,96.10,101.00,...,61.90,44.80,36.50,31.00,29.00,25.10,26.90,22.40,28.10,18.90
978,8호선,복정,송파구,2821,하선,주말,6.05,5.95,8.35,7.85,...,20.90,24.05,20.55,18.60,19.45,20.20,19.45,21.45,21.50,19.00


#### aircondition 추가작업

In [235]:
# aircondition과 passenger 병합

aircondition_merged = pd.merge(station, aircondition, on = ['호선', '역명'])
merged_df = pd.merge(aircondition_merged, passengers, on = ['역번호'])

In [239]:
# 중복되는 열 제거

merged_df.drop(columns = ['호선_y', '역명_y',], inplace = True)

# 열이름 변경 
merged_df.columns = ['호선', '역명'] + list(merged_df.columns[2:])

merged_df.head()

,호선,역명,자치구,역번호,미세먼지,이산화탄소,포름알데히드,일산화탄소,공기질점수,1월,...,4월,5월,6월,7월,8월,9월,10월,11월,12월,승객수
0,1호선,서울역,중구,150,0.335559,0.623145,0.371166,0.166667,0.408431,3198290.0,...,1803550.0,2112629.0,2201148.0,2391615.0,1986290.0,1705059.0,2121997.0,2218093.0,1648262.0,0.478710
1,1호선,시청,중구,151,0.741235,0.572700,0.414110,0.500000,0.700221,1477362.0,...,874072.0,962697.0,1042088.0,1134098.0,922958.0,853911.0,994423.0,1072922.0,857434.0,0.225062
2,1호선,종각,종로구,152,0.726210,0.801187,0.496933,0.416667,0.785164,2462560.0,...,1586710.0,1745454.0,1892509.0,2069902.0,1626359.0,1378130.0,1702997.0,1758055.0,1314449.0,0.394990
3,1호선,종로3가,종로구,153,0.818030,0.679525,0.500000,0.750000,0.907447,1935106.0,...,1224282.0,1374997.0,1360814.0,1523797.0,1212158.0,1051916.0,1298918.0,1297683.0,953894.0,0.293588
4,1호선,종로5가,종로구,154,0.719533,0.341246,0.371166,0.583333,0.615350,1527217.0,...,1040487.0,1182917.0,1182427.0,1263489.0,1017196.0,988661.0,1124400.0,1100466.0,930913.0,0.248899


#### merge_df 엑셀로 보내기 

In [240]:
merged_df.to_csv('Data/Found Data/공기질_승객수_통합.csv', encoding = 'euc-kr')

#### 좌표 데이터와 기존 데이터 병합

In [247]:
subway = pd.read_csv('Data/Found Data/좌표.csv', header = 0, index_col = 0)
subway.head()

,위도,경도
가락시장,37.492522,127.118234
종로3가,37.571607,126.991806
오금,37.502162,127.128111
동대문,37.571420,127.009745
동대문역사문화공원,37.565138,127.007896


In [248]:
subway['역명'] = subway.index
subway = pd.merge(merged_df, subway, on = ['역명'])
subway.head()

,호선,역명,자치구,역번호,미세먼지,이산화탄소,포름알데히드,일산화탄소,공기질점수,1월,...,6월,7월,8월,9월,10월,11월,12월,승객수,위도,경도
0,1호선,시청,중구,151,0.741235,0.572700,0.414110,0.500000,0.700221,1477362.0,...,1042088.0,1134098.0,922958.0,853911.0,994423.0,1072922.0,857434.0,0.225062,37.564718,126.977108
1,2호선,시청,중구,201,0.884808,0.418398,0.174847,0.500000,0.600502,1410239.0,...,1126264.0,1205606.0,980247.0,924240.0,1031394.0,1124381.0,911876.0,0.238443,37.564718,126.977108
2,1호선,종각,종로구,152,0.726210,0.801187,0.496933,0.416667,0.785164,2462560.0,...,1892509.0,2069902.0,1626359.0,1378130.0,1702997.0,1758055.0,1314449.0,0.394990,37.570161,126.982923
3,1호선,종로3가,종로구,153,0.818030,0.679525,0.500000,0.750000,0.907447,1935106.0,...,1360814.0,1523797.0,1212158.0,1051916.0,1298918.0,1297683.0,953894.0,0.293588,37.571607,126.991806
4,3호선,종로3가,종로구,319,0.864775,0.587537,0.294479,0.500000,0.707698,714572.0,...,556592.0,616889.0,482819.0,448247.0,525569.0,540538.0,429329.0,0.111448,37.571607,126.991806
